In [1]:
import sys
sys.path.insert(0,'/home/max/ironyun_proj/color-detection')

from version.v_1.Color_Search import *
from search_color import *
from colorEval import *
# import the necessary packages
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import sys, os
import time
from PIL import Image

In [2]:
def inRange(img, lower_b, upper_b):
    ch1, ch2, ch3 = cv2.split(img)
    ch1m = (lower_b[0] <= ch1) & (ch1 <= upper_b[0])
    ch2m = (lower_b[1] <= ch2) & (ch2 <= upper_b[1])
    ch3m = (lower_b[2] <= ch3) & (ch3 <= upper_b[2])
    mask = ch1m & ch2m & ch3m
    return mask.astype(np.uint8)*255

def imgFilt(frame,filtratio):

    
    #filtered_img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    grimg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    hsvimg = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    lightMask = np.asarray(grimg)
    Mask_indices = lightMask > 255 * filtratio
    frame[Mask_indices] =255
    return (frame, Mask_indices)

def imgHsvFilt(img):

    filtered_img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    #grimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2GRAY)

    hsvimg = cv2.cvtColor(filtered_img,cv2.COLOR_BGR2HSV)

    lightMask = np.asarray(hsvimg)
    Mask_indices = lightMask > 150
    filtered_img[Mask_indices] =255
    return filtered_img

def white_balance(frame_in):

    # Gray World Plus Retinex Theory for Automatic White Balance

    pimg = cv2.cvtColor(frame_in, cv2.COLOR_BGR2RGB)
    pimg = Image.fromarray(pimg)
    pimg = pimg.convert(mode='RGB')
    nimg = np.asarray(pimg)
    nimg.flags.writeable = True

    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = nimg[1].max()
    nimg[0] = np.minimum(nimg[0] * (mu_g / float(nimg[0].max())),255)
    nimg[2] = np.minimum(nimg[2] * (mu_g / float(nimg[2].max())),255)
    nimg = nimg.transpose(1, 2, 0).astype(np.uint8)

    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    sum_r = np.sum(nimg[0])
    sum_r2 = np.sum(nimg[0] ** 2)
    max_r = nimg[0].max()
    max_r2 = max_r ** 2
    sum_g = np.sum(nimg[1])
    max_g = nimg[1].max()
    coefficient = np.linalg.solve(np.array([[sum_r2, sum_r], [max_r2, max_r]]), np.array([sum_g, max_g]))

    nimg[0] = np.minimum(nimg[0] ** 2 * coefficient[0] + nimg[0] * coefficient[1], 255)
    sum_b = np.sum(nimg[1])
    sum_b2 = np.sum(nimg[1] ** 2)
    max_b = nimg[1].max()
    max_b2 = max_r ** 2
    coefficient = np.linalg.solve(np.array([[sum_b2, sum_b],[max_b2, max_b]]), np.array([sum_g, max_g]))
    nimg[1] = np.minimum(nimg[1] ** 2 * coefficient[0] + nimg[1] * coefficient[1], 255)

    nimg = nimg.transpose(1, 2, 0).astype(np.uint8)

    rgb_nimg = Image.fromarray(np.uint8(nimg)).convert('RGB')
    awb_img = np.array(rgb_nimg)
    awb_img = awb_img[:, :, ::-1].copy()
    
    return awb_img

def crop_aspect_ratio(img, isUpper, aspect_ratio = 0.5):

    src_height, src_width, src_channels = img.shape

    roix_upper = int(src_width * 0.25)
    roix_upper_width = int(src_width * aspect_ratio)
    roiy_upper = int(src_height * 0.15)
    roiy_upper_width = int(src_height * 0.35)

    cropped_img = img[ roiy_upper: roiy_upper + roiy_upper_width,roix_upper: roix_upper + roix_upper_width]


    return cropped_img

In [8]:
%matplotlib inline

#  Log system        
class test_Search_Color:

    def color_detection(self, frame_in, color2opt, roiScale, filtratio, color_param, detect_type = 'car'):
        
        if detect_type == 'person':
            frame_in = crop_aspect_ratio(frame_in,True)
        
        frame_in = white_balance(frame_in)

        roiWid = 10
        roiEdg = roiScale
        
        
        src_height, src_width, src_channels = frame_in.shape
        
        roiX = int(src_width / roiWid)
        roiWidth = int(roiX * roiEdg)
        roiY = int(src_height / roiWid)
        roiHeight = int(roiY * roiEdg)
        
        frame = frame_in#[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
            
        if detect_type == 'car':
            frame,lightMask = imgFilt(frame,filtratio)
            blockedPxl = sum(np.count_nonzero(e) for e in lightMask)
        else:
            blockedPxl = 0

        
        src_height, src_width, src_channels = frame.shape
        
        max_value = ((src_height*src_width)-blockedPxl)*255

    
        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        
        if color2opt is 'white':
            lower = np.array([0,0, color_param[0]])   
            upper = np.array([179, color_param[1], 255])
            mask = cv2.inRange(hsv, lower, upper)
            lower = np.array([0,color_param[1], color_param[2]])
            upper = np.array([179, color_param[3], 255])
            mask = mask + cv2.inRange(hsv, lower, upper)
        
        elif color2opt is 'black':
            lower = np.array([0,0,0])
            upper = np.array([179,color_param[0],color_param[1]])
            mask  = cv2.inRange(hsv,lower,upper)
            lower = np.array([0,color_param[0],0])
            upper = np.array([179,255,color_param[2]])
            mask  = mask + cv2.inRange(hsv, lower, upper)
        
        else: #others
            lower = np.array([143,37, 20])   
            upper = np.array([210, 255, 255])
            mask = cv2.inRange(hsv, lower, upper)
        
        if(float(max_value) != 0):
            Val = float(mask.sum()) / float(max_value)
        else:
            Val = 0
    
        res = round(Val,2)

        return res

In [9]:
#search_color = PS_Search_Color()
def opt_color(threshold, roiScale, filtratio, color_param, color2opt):

    clr_idx = colorset.index(color2opt)

    color_to_be_tested = ['red','yellow','blue','green','white','black']
    target_folder = 'color_data/human/upper/'
    this_color_count = 0
    fp = 0
    fn = 0
    #ps_count = 0
    total = 0
    for idx, color in enumerate(color_to_be_tested):
        for file in os.listdir(target_folder + color):
            if(file != os.path.basename("__file__")):
                if(file[-3:] == 'jpg'):
                    
                    origin_img = cv2.imread(target_folder + color + '/' + file)

                    res = test_search_color.color_detection(origin_img, color2opt, roiScale, filtratio, color_param, 'person')

                    total += 1

                    if res>=threshold and clr_idx != idx:
                        fp = fp +1
                    elif clr_idx == idx:
                        this_color_count += 1
                        if res<threshold:
                            fn = fn + 1

    return total,this_color_count,fp,fn

In [10]:
#config.py
from hyperopt import hp
base_space = [
    hp.uniform('threshold',0,10), #threshold of detection      
    hp.uniform('roiScale',4,10), #roi scale   
    hp.uniform('filtratio',0.5,0.75),  #mask filter threshold
    
]
color_space_dict = {
    'color_space_black' : [  
        hp.uniform('black_h_lower',15,45),
        hp.uniform('black_h_upper',50,80),
        hp.uniform('black_s_lower', 60,120)       
    ],
    
    'color_space_red' : [ 
        hp.uniform('red_h_lower',140,160),
        hp.uniform('red_h_upper',200,220),
        hp.uniform('red_s_lower',  30,80),
        hp.uniform('red_v_lower',20,100),
    ],
    'color_space_yellow' : [
        hp.uniform('green_h_lower',18,25),
        hp.uniform('green_h_upper',35,75),
        hp.uniform('yellow_s_lower', 30,80),
        hp.uniform('yellow_v_lower',20,100),
    ],
    'color_space_blue' : [
        hp.uniform('blue_h_lower',70,90),
        hp.uniform('blue_h_upper',120,140),
        hp.uniform('blue_s_lower', 30,80),
        hp.uniform('blue_v_lower',20,100),
    ],
    'color_space_green' : [
        hp.uniform('green_h_lower',36,50),
        hp.uniform('green_h_upper',70,115),
        hp.uniform('green_s_lower', 30,80),
        hp.uniform('green_v_lower',20,100),
    ],
    'color_space_white' : [
        hp.uniform('white_h_lower',170,210),
        hp.uniform('white_h_upper',0,20),
        hp.uniform('white_s_lower', 160,210),
        hp.uniform('white_v_lower',15,60),
    ]   
}

colorset    = ['red','yellow','blue','green','white','black',]
color2opt   = ['red','yellow','blue','green','white','black',]
log_prefix         = 'res_log.txt'


In [11]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pprint import pprint
import hyperopt.pyll.stochastic
from __future__ import division
# minimize the objective over the space
from hyperopt import fmin, tpe
import hyperopt
import sys
import time

min = 1
state = ''
# define an objective function
def objective(args):
    global min
    global state
    global color
    global log
    
    
    threshold, roiScale, filtratio = args[0:3]
    color_param = args[3:]
    color_param = [int(param) for param in color_param]
    
    thr = 0.1 + threshold * 0.05
    
    total,this_color_count,fp,fn = opt_color(thr, roiScale, filtratio, color_param,color)
    tp = total - fp - fn
    beta = 10
    
    f1_score = float(((1 + beta**2) * tp)) / ((1 + beta**2)*tp + beta**2 * fn + fp)
    
    if(1 - f1_score < min):
        min = 1 - f1_score
        state = {
            'color'      :   color,
            'Total_frame':   total,
            'Color_frame':   this_color_count,
            'f1_score':      round(1-f1_score,3),
            'fp':            round(fp/(total-this_color_count),3),
            'fn':            round(fn/this_color_count,3),
            'thr':           thr,
            'roiScale':      roiScale,
            'filtratio':     filtratio,
            'color_param':   color_param,
            
        }
        with open(color + '_' + log_prefix, "a") as res_log:
            pprint(state, res_log)
        pprint(state)
        print('\n')

    return {
        'loss': 1-f1_score,
        'status': STATUS_OK,
        'eval_time':time.time()
    }
 
test_search_color = test_Search_Color()

for color in color2opt:
    if color in colorset:
        print color
        
        space = base_space + color_space_dict['color_space_' + color]

        #pprint(trials.trials)
        print('\n')
    
        trials = Trials()
        best = fmin(objective, space, algo=tpe.suggest, max_evals=6000,trials= trials)

        pprint(best)
        print('\n')

        pprint(trials.losses())
        #pprint(hyperopt.space_eval(space, best))
        print('\n')

red


{'Color_frame': 224,
 'Total_frame': 1125,
 'color': 'red',
 'color_param': [145, 213, 60, 91],
 'f1_score': 0.161,
 'filtratio': 0.5815781925447898,
 'fn': 0.817,
 'fp': 0.0,
 'roiScale': 7.82086570298828,
 'thr': 0.5604935790352062}


{'Color_frame': 224,
 'Total_frame': 1125,
 'color': 'red',
 'color_param': [147, 202, 78, 29],
 'f1_score': 0.061,
 'filtratio': 0.6192491374190617,
 'fn': 0.304,
 'fp': 0.019,
 'roiScale': 5.750731365468798,
 'thr': 0.1815912107109535}




KeyboardInterrupt: 